# Experiment 1:

- Our base sketch transformer architecture
- Single class
- DeltaPenPositionTokenizer

__additional__
- Post processing (bezier inference)
- Sketch completion
- Perplexity, curve count hueristic

In [ ]:
import os

os.chdir("..")

In [ ]:
import torch
import torch.nn.functional as F
from dataset import QuickDrawDataset
from tokenizers import DeltaPenPositionTokenizer
from models import SketchTransformer
from runner import SketchTrainer, sample
from prepare_data import stroke_to_bezier_single, clean_svg

dataset = QuickDrawDataset(label_names=["cat"], download=True)
tokenizer = DeltaPenPositionTokenizer(bins=32)

# d_model => model capacity (types of drawing features it can learn)
# nhead => model can attend to more positions in parallel
# num layers => model learns more hierarchical abstractions (patterns, shapes, layouts)

model = SketchTransformer(
    vocab_size=len(tokenizer.vocab), d_model=384, nhead=8, num_layers=8, max_len=200
)

training_config = {
    "batch_size": 128,
    "num_epochs": 15,
    "learning_rate": 1e-4,
    "log_dir": "logs/sketch_transformer_experiment_1",
    "resume": True,
    "splits": [0.85, 0.1, 0.05],
    # "use_padding_mask": True,
}

trainer = SketchTrainer(model, dataset, tokenizer, training_config)

In [ ]:
trainer.train_mixed(training_config["num_epochs"])

In [ ]:
generations_inline = ""
generations = []

for i in range(5):
    generated = sample(
        model=trainer.model,
        start_tokens=[trainer.tokenizer.vocab["START"]],
        temperature=1.0,
        top_k=20,
        top_p=0.7,
        greedy=False,
        eos_id=trainer.tokenizer.vocab["END"],
    )
    decoded_sketch = tokenizer.decode(generated, stroke_width=0.3)
    decoded_sketch = stroke_to_bezier_single(decoded_sketch)
    decoded_sketch = clean_svg(decoded_sketch)

    generations_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Generated</b><br>{decoded_sketch}</div>'
    generations.append((generated, decoded_sketch))


from IPython.display import HTML, display

display(HTML(generations_inline))

# temp<=0.5 fairly deterministic
# temp=0.8, top_k=20, top_p=0.9 more variety but still coherent

# *note important features are usually preseved, but sketches are disorganized (number of curves hueristic does not work well)*
# temp=1.0, top_k=20, top_p=0.75 more variety, some incoherent sequences

#  *note that lower temp means less variety, notice that sequences begin to repete themselves more often*
# temp=0.55, top_k=20, top_p=0.9 good balance
# temp=0.6, top_k=30, top_p=0.9  good balance

In [ ]:
# note many sketches have missing parts or incomplete shapes (step 1: get a base sketch) : check the number of paths
# psuedo hueuristic: count number of curves in SVG

from prepare_data import count_curves

# sort generations by number of curves
generations_inline = ""

generations_sorted = sorted(generations, key=lambda x: count_curves(x[1]), reverse=True)
for sketch in generations_sorted:
    generations_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Generated</b><br>{sketch[1]}</div>'

display(HTML(generations_inline))

# Sketch completion

Visualization of how the model has learned the underlying representation of probable sketches for a given label.

Sort items from the dataset based on perplexity.

In [ ]:
# Select a sketch from the dataset, remove tokens and let the model complete it
selected_sketch = dataset[38946]

# tokenize and remove some tokens from the end
selected_tokens = tokenizer.encode(selected_sketch)

selected_tokens_partial = selected_tokens[: len(selected_tokens) // 2]  # remove 50%
destroyed_sketch = tokenizer.decode(selected_tokens_partial)

comparison_inline = f"""<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Original</b><br>{selected_sketch}</div>
<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Partial</b><br>{destroyed_sketch}</div>"""

for i in range(5):
    generated = sample(
        model=trainer.model,
        start_tokens=selected_tokens_partial,
        temperature=1.0,
        greedy=False,
        top_k=20,
        top_p=0.9,
        eos_id=trainer.tokenizer.vocab["END"],
    )

    generated_sketch = tokenizer.decode(generated, stroke_width=0.3)
    generated_sketch = stroke_to_bezier_single(generated_sketch)
    generated_sketch = clean_svg(generated_sketch)
    comparison_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Completed {i}</b><br>{generated_sketch}</div>'

display(HTML(comparison_inline))

## Sketch representation

Visualization of how the model has learned the underlying representation of probable sketches for a given label.

Sort items from the dataset based on perplexity.

In [ ]:
def compute_perplexity(model, tokens):
    model.eval()
    with torch.no_grad():
        logits = model(tokens[:, :-1])
        target = tokens[:, 1:]
        loss = F.cross_entropy(
            logits.transpose(1, 2),  # (batch, vocab, seq_len)
            target,
            reduction="none",
        )

        loss = loss.mean(dim=1)
        perplexity = torch.exp(loss)
        return perplexity


sketch_perplexities = []

for i in range(20):
    sketch = dataset[i]
    tokens = tokenizer.encode(sketch)
    perplexity = compute_perplexity(model, torch.tensor([tokens]))
    decoded_sketch = tokenizer.decode(tokens, stroke_width=0.3)
    sketch_perplexities.append((perplexity.item(), decoded_sketch))

# sort by perplexity
sketch_perplexities.sort(key=lambda x: x[0], reverse=True)

# sort normalized by length
# sketch_perplexities.sort(key=lambda x: x[0] / len(x[1]), reverse=True)

sketches_inline = ""
for perp, sketch in sketch_perplexities:
    sketches_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Perplexity: {perp:.2f}</b><br>{sketch}</div>'


from IPython.display import HTML, display

display(HTML(sketches_inline))

# Sorting by perplexity does seem to highlight some of the worse sketches

Computing perplexities: 100%|██████████| 20/20 [00:00<00:00, 57.16it/s]
